# Creative Writing Workbench using SequentialChain (version 2)

Solution to **exercise #6**

Convert the Creative Writing Workbench code from *exercise #5* to use LangChain SequentialChain.


Create a SequentialChain with 4 LLMChain(s)

1. LLMChain 1 : Generate 3 plan
2. LLMChain 2 : Vote the best plan out of the 3 plans
3. LLMChain 3 : Generate 3 writeups from the best plan
4. LLMChain 4 : Vote the best passage out of the 3 passages

To keep things simple you can us the same LLM for all chains.

**Hints**:

* To specify JSON formats in prompts use double '{{' instead of '{'  i.e., *{{   ...  }*
* You will need to adjust the prompt i.e., you cant use the prompts from earlier version as-is
* You may need to adjust the number of tokens returned from the model as a response
* Use JSON output parser. Pydantic may also be used. Try out both and pick one that works best for you.
* Output indicators may be hard coded in the prompt as we will not be parsing the output from the LLM at each step

In [1]:
from dotenv import load_dotenv
import os

import warnings

from IPython.display import Markdown, JSON

warnings.filterwarnings("ignore")

# Load the file that contains the API keys
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

True

In [2]:
writeup_instructions = """
Write a passage of under 100 words to describe our new product:
Javelin 20 High Performance Shoes.

The title must contain the sentence: A new you

The first sentence must contain a few of these words: motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : trendsetter
"""

In [3]:
### from langchain.chat_models import ChatOpenAI
# from langchain_openai import ChatOpenAI

### model_name = "gpt-3.5-turbo"
# llm = ChatOpenAI(temperature=0.4) #model_name=model_name)

In [4]:
# from langchain.llms import Cohere

from langchain_community.llms import Cohere

llm = Cohere(model="command", max_tokens=3000)


## 1. Create LLMChain #1   Generate 3 plans


In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains  import LLMChain, SequentialChain

from IPython.display import Markdown, JSON

In [6]:
ouput_passage_indicator = "Passage:"
ouput_plan_indicator = "Plan:"

template_1 = """
Generate three separate step by step plans for executing the given task with the following task instructions.

Task Instructions:
{task_instructions}

{ouput_plan_indicator}
Your plan here.

""" 

prompt_template_1 = PromptTemplate(
                        template = template_1,
                        input_variables = ["task_instructions"],
                        partial_variables = {"ouput_plan_indicator":ouput_plan_indicator}
)

print(prompt_template_1.format(task_instructions=writeup_instructions))


Generate three separate step by step plans for executing the given task with the following task instructions.

Task Instructions:

Write a passage of under 100 words to describe our new product:
Javelin 20 High Performance Shoes.

The title must contain the sentence: A new you

The first sentence must contain a few of these words: motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : trendsetter


Plan:
Your plan here.




In [7]:
# Create the LLM Chain
llm_chain_generate_plan = LLMChain(
    prompt = prompt_template_1,
    llm = llm,
    output_key = 'proposed_plans',
)

response = llm_chain_generate_plan.invoke({"task_instructions": writeup_instructions})

In [8]:
# print(response['proposed_plans'])

## 2. Vote the best plan out of the 3 plans

In [9]:
output_best_plan_indicator = "Selected Plan:"

template_2 = """
Three  expert marketing writers were asked to put together a plan for the following task. 
Your job is to pick the best plan.

Task Instructions:
{task_instructions}

Proposed Plans:
{proposed_plans}

Your out should be in following format.

{output_best_plan_indicator} 
Put the plan steps here.

"""

prompt_template_2 = PromptTemplate(
                        template = template_2,
                        input_variables = ["task_instructions", "proposed_plans"],
                        partial_variables = {"output_best_plan_indicator":output_best_plan_indicator}
)

print(prompt_template_2.format(task_instructions=writeup_instructions, proposed_plans = "DUMMY PLANS"))


Three  expert marketing writers were asked to put together a plan for the following task. 
Your job is to pick the best plan.

Task Instructions:

Write a passage of under 100 words to describe our new product:
Javelin 20 High Performance Shoes.

The title must contain the sentence: A new you

The first sentence must contain a few of these words: motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : trendsetter


Proposed Plans:
DUMMY PLANS

Your out should be in following format.

Selected Plan: 
Put the plan steps here.




In [10]:
# Create the LLM Chain
llm_chain_vote_on_plan = LLMChain(
    prompt = prompt_template_2,
    llm = llm,
    output_key="best_voted_plan",
)


## Intermediate Sequential Chain to test

OPTIONAL step - just to check how things are going

In [11]:
from langchain.chains import SequentialChain


# {"task_instructions": writeup_instructions}

sequential_chain = SequentialChain(
    chains=[llm_chain_generate_plan, llm_chain_vote_on_plan],
    input_variables = ['task_instructions', ],
    output_variables = ["best_voted_plan", "proposed_plans"],
    verbose = True
)

In [12]:
response = sequential_chain.invoke({"task_instructions":writeup_instructions})



> Entering new SequentialChain chain...

> Finished chain.


In [13]:
# JSON(response) #['best_voted_plan'])

## 3 Create 3 passages with best_voted_plan

In [14]:
template_3 = """
Create three passages using the following task instructions and the plan.

Task Instructions:
{task_instructions}

Plan:
{best_voted_plan}

Your output will be in following format.

{ouput_passage_indicator}
Put the passage here.

"""

prompt_template_3 = PromptTemplate(
    template = template_3,
    input_variables = ["best_voted_plan"],
    partial_variables = {"ouput_passage_indicator" : ouput_passage_indicator, "task_instructions": writeup_instructions},
)

# print(prompt_template_3.format(best_voted_plan="DUMMY PLAN"))

In [15]:
# Create the LLM Chain
llm_chain_generate_passages = LLMChain(
    prompt = prompt_template_3,
    llm = llm,
    output_key="passages",
)

## CREATE Intermediate Sequential Chain

OPTIONAL step - just to check how things are going


In [16]:
sequential_chain = SequentialChain(
    chains=[llm_chain_generate_plan, llm_chain_vote_on_plan, llm_chain_generate_passages],
    input_variables = ['task_instructions', ],
    output_variables = ["passages","best_voted_plan","proposed_plans", "passages"],
    verbose = True
)

In [17]:
response = sequential_chain.invoke({"task_instructions":writeup_instructions})



> Entering new SequentialChain chain...

> Finished chain.


In [18]:
JSON(response)

<IPython.core.display.JSON object>

## 4 Vote the best passage

In [19]:

output_best_passage_indicator = "Selected Passage:"

template_4 = """
Three experts were given the following task.

Task:
{task_instructions}

You need to review the 3 passages and identify ONE best passage out of the three. 

Proposed Passages:
{passages}

Your output will be in following format.

{output_best_passage_indicator}
Put the passage here.

"""

prompt_template_4 = PromptTemplate(
    template = template_4,
    input_variables = ["passages"],
    partial_variables = {"task_instructions": writeup_instructions, "output_best_passage_indicator" : output_best_passage_indicator, },
)

print(prompt_template_4.format(passages = "DUMMY PASSAGES"))


Three experts were given the following task.

Task:

Write a passage of under 100 words to describe our new product:
Javelin 20 High Performance Shoes.

The title must contain the sentence: A new you

The first sentence must contain a few of these words: motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : trendsetter


You need to review the 3 passages and identify ONE best passage out of the three. 

Proposed Passages:
DUMMY PASSAGES

Your output will be in following format.

Selected Passage:
Put the passage here.




In [20]:
# Create the LLM Chain
llm_chain_select_final_passages = LLMChain(
    prompt = prompt_template_4,
    llm = llm,
    output_key="creative_passage",
)

In [21]:
sequential_chain = SequentialChain(
    chains=[llm_chain_generate_plan, llm_chain_vote_on_plan, llm_chain_generate_passages, llm_chain_select_final_passages],
    input_variables = ['task_instructions', ],
    output_variables = ["creative_passage", "best_voted_plan", "passages", "proposed_plans"],
    verbose = True,
)

In [ ]:
response = sequential_chain.invoke({"task_instructions":writeup_instructions})



> Entering new SequentialChain chain...


In [ ]:
JSON(response)